In [1]:
import pandas as pd
import numpy as np
from pandas import Series, DataFrame
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt 
import statsmodels.formula.api as smf
import statsmodels
from sklearn.linear_model import LinearRegression

In [2]:
# データ読み込み
X = pd.read_csv("test.csv")

In [3]:
# 明らかに関係ない値や欠損値が１万を超えるものを含む列を削る
delete_col_list = ["avgsr4","avgWin4","dhweight", "disavesr", 'hweight',
       'jwinper','owinper', 'surface',"disRoc","eps","winRun","preOOF","runningStyle","lateStartPer",
        'twinper',  'jEps',"dsl","preSRa",
    'month', 'ridingStrongJockey',"jAvgWin4",
       'preLateStart', 'preLastPhase', "surfaceScore","pre2OOF",
       'course', 'placeCode', 'headCount',"preHeadCount","surfaceChanged",
        "gradeChanged","preMargin","femaleOnly"]

# delete_col_listで指定した列名をデータから削除する。
X = X.drop(delete_col_list,axis=1)

In [11]:
# 元のデータセットを汚さないように一応コピー
df = X.copy()

#  order_of_finishにNaNを含む行を抽出
df_rn=df[df["order_of_finish"].isnull()]

# 欠損値補間用に空のデータフレーム作成
list_df = pd.DataFrame( columns=['race_id', 'horse_number', 'grade', 'order_of_finish', 'horse_id',
       'jockey_id', 'trainer_id', 'age', 'distance', 'enterTimes', 'odds',
       'sex', 'weather', 'weight'] )

# order_of_finishが0である行のrace_idをforでまわし、各レースにごとに取り出して、欠損値を補間
for i in list(df_rn["race_id"].unique()):
    # 同じrace_idで構成されたtmp_df
    tmp_df=df[df["race_id"]==i]
    # あるレース中に欠損値が一つなら自明なのでそれを代入
    if(tmp_df["order_of_finish"].isnull().sum()==1):
        # horse_numbserの合計とorder_of_finishの合計が一致するはずなのでその差が順位
        fill_order = tmp_df["horse_number"].sum()-tmp_df["order_of_finish"].sum()
        list_df = list_df.append(tmp_df.fillna(fill_order))
    else:
        # 2個以上あるときは既知の順位の中でもっとも遅いものに２足したもので埋める(データを見たら最後の方が抜けてることが多かったため)
        list_df = list_df.append(tmp_df.fillna({"order_of_finish":max(tmp_df["order_of_finish"])+2}))
list_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5659 entries, 53 to 52685
Data columns (total 14 columns):
race_id            5659 non-null object
horse_number       5659 non-null object
grade              5659 non-null object
order_of_finish    5659 non-null float64
horse_id           5659 non-null object
jockey_id          5659 non-null object
trainer_id         5659 non-null object
age                5659 non-null float64
distance           5659 non-null float64
enterTimes         5659 non-null float64
odds               5634 non-null float64
sex                5659 non-null object
weather            5659 non-null object
weight             5659 non-null float64
dtypes: float64(6), object(8)
memory usage: 663.2+ KB


In [50]:

#Xからorder_of_finishにnullを含む行と同じrace_idを持つ行全てを削る、その後先程作った行を足す。
for i in list(df_rn["race_id"].unique()):
    df=df[df["race_id"] != i]

# order_of_finishに欠損値があるため削った分の行を足す
X_filled = df.append(list_df)

# oddsを中央値で埋める
X_filled = X_filled.fillna(X_filled.median())

# 文字列をダミー変数に変換
X_filled_dammy = pd.get_dummies(X_filled)

# 目的変数
y = X_filled_dammy.order_of_finish

# 説明変数
x=X_filled_dammy.drop("order_of_finish", axis=1)

# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=3)

In [56]:
stdsc = StandardScaler()
# 訓練用のデータを標準化

x_std = stdsc.fit_transform(x)

x_std_pd = pd.DataFrame(stdsc.fit_transform(x))
# x_std.columns = list(x.columns)

AttributeError: 'numpy.ndarray' object has no attribute 'columns'

In [71]:
#　標準化した際に、numpy.arrayになってしまい、列名が飛ぶ。
# pandas.DataFrameを入力にするとｙとxのindexがnot aligned って出る
# 調整中
model = smf.OLS(y,x_std)
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        order_of_finish   R-squared:                       0.059
Model:                            OLS   Adj. R-squared:                  0.058
Method:                 Least Squares   F-statistic:                     184.9
Date:                Sun, 17 Feb 2019   Prob (F-statistic):               0.00
Time:                        00:15:38   Log-Likelihood:            -1.9117e+05
No. Observations:               53354   AIC:                         3.824e+05
Df Residuals:                   53336   BIC:                         3.825e+05
Df Model:                          18                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1            -0.0351      0.038     -0.930      0.352      -0.109       0.039
x2             0.2656      0.038      7.004      0.000       0.191       0.340
x3             0.0807      0.050      1.608      0.108      -0.018       0.179
x4             0.2049      0.190      1.076      0.282      -0.168       0.578
x5            -0.3077      0.038     -8.024      0.000      -0.383      -0.233
x6            -0.0071      0.038     -0.186      0.852      -0.081       0.067
x7             0.5652      0.192      2.940      0.003       0.188       0.942
x8            -0.2254      0.041     -5.535      0.000      -0.305      -0.146
x9            -0.2514      0.040     -6.209      0.000      -0.331      -0.172
x10            1.9770      0.039     51.186      0.000       1.901       2.053
x11           -0.1533      0.053     -2.872      0.004      -0.258      -0.049
x12            0.0346      0.036      0.958      0.338      -0.036       0.105
x13            0.0290      0.026      1.110      0.267      -0.022       0.080
x14           -0.0428      0.024     -1.800      0.072      -0.089       0.004
x15           -0.0167      0.038     -0.443      0.658      -0.090       0.057
x16            0.0351      0.036      0.986      0.324      -0.035       0.105
x17           -0.0140      0.020     -0.685      0.494      -0.054       0.026
x18            0.0076      0.023      0.329      0.742      -0.038       0.053
x19           -0.0112      0.035     -0.318      0.750      -0.080       0.058
x20           -0.0006      0.038     -0.016      0.987      -0.075       0.073
==============================================================================
Omnibus:                     2689.337   Durbin-Watson:                   0.194
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1765.519
Skew:                           0.326   Prob(JB):                         0.00
Kurtosis:                       2.393   Cond. No.                     1.82e+15
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 5.27e-26. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [18]:
tmp_df=df[df["race_id"]==5]
tmp_df["horse_number"].sum()-tmp_df["order_of_finish"].sum()

16.0

In [35]:
x_std.columns = x.columns

In [55]:
x_std_pd.columns

RangeIndex(start=0, stop=20, step=1)

In [34]:
x.columns

Index([u'race_id', u'horse_number', u'grade', u'horse_id', u'jockey_id',
       u'trainer_id', u'age', u'distance', u'enterTimes', u'odds', u'weight',
       u'sex_セ', u'sex_牝', u'sex_牡', u'weather_他', u'weather_小雨', u'weather_晴',
       u'weather_曇', u'weather_雨', u'weather_雪'],
      dtype='object')

In [65]:
x_std.shape

(53354, 20)

In [66]:
x_std_pd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53354 entries, 0 to 53353
Data columns (total 20 columns):
0     53354 non-null float64
1     53354 non-null float64
2     53354 non-null float64
3     53354 non-null float64
4     53354 non-null float64
5     53354 non-null float64
6     53354 non-null float64
7     53354 non-null float64
8     53354 non-null float64
9     53354 non-null float64
10    53354 non-null float64
11    53354 non-null float64
12    53354 non-null float64
13    53354 non-null float64
14    53354 non-null float64
15    53354 non-null float64
16    53354 non-null float64
17    53354 non-null float64
18    53354 non-null float64
19    53354 non-null float64
dtypes: float64(20)
memory usage: 8.1 MB


In [43]:
y.isnull().sum()

0

In [44]:
x_std.isnull().sum()

race_id         0
horse_number    0
grade           0
horse_id        0
jockey_id       0
trainer_id      0
age             0
distance        0
enterTimes      0
odds            0
weight          0
sex_セ           0
sex_牝           0
sex_牡           0
weather_他       0
weather_小雨      0
weather_晴       0
weather_曇       0
weather_雨       0
weather_雪       0
dtype: int64

In [69]:
x

,race_id,horse_number,grade,horse_id,jockey_id,trainer_id,age,distance,enterTimes,odds,weight,sex_セ,sex_牝,sex_牡,weather_他,weather_小雨,weather_晴,weather_曇,weather_雨,weather_雪
0,1,9,3,2014105125,700,1033,4.0,2000.0,1.0,8.2,57.0,0,0,1,0,0,0,0,1,0
1,1,5,3,2015103024,1154,1002,3.0,2000.0,7.0,93.7,55.0,0,0,1,0,0,0,0,1,0
2,1,8,3,2015101580,5203,1058,3.0,2000.0,8.0,3.1,55.0,0,0,1,0,0,0,0,1,0
3,1,2,3,2015104823,1115,1039,3.0,2000.0,2.0,2.7,53.0,0,1,0,0,0,0,0,1,0
4,1,10,3,2015102906,1102,425,3.0,2000.0,8.0,3.8,55.0,0,0,1,0,0,0,0,1,0
5,1,12,3,2013103153,1088,1046,5.0,2000.0,6.0,21.7,57.0,0,0,1,0,0,0,0,1,0
6,1,6,3,2015104257,1116,1104,3.0,2000.0,6.0,11.3,53.0,0,1,0,0,0,0,0,1,0
7,1,3,3,2015100839,1138,378,3.0,2000.0,7.0,287.0,53.0,0,1,0,0,0,0,0,1,0
8,1,1,3,2014102931,1171,1083,4.0,2000.0,3.0,140.0,54.0,0,0,1,0,0,0,0,1,0
9,1,7,3,2015100716,1168,1110,3.0,2000.0,2.0,399.6,51.0,0,1,0,0,0,0,0,1,0
